# Data Cleaning & Merging – Hilal and Dubizzle Real Estate

## 1. Load Required Libraries

In [1]:
import pandas as pd
import numpy as np

## 2. Load the CSV Files

In [2]:
hilal_df = pd.read_csv("Desktop/hilal_rental_data.csv")
dubizzle_df = pd.read_csv("Desktop/dubizzle_properties_for_rent.csv")

## 3. Explore Raw Data

In [3]:
print("Hilal Dataset:")
hilal_df.head()

Hilal Dataset:


,Title,Location,Bedrooms,Bathrooms,Price,Size,Listing_Type
0,2-BEDROOM APARTMENT,Al Ansab,2,2,300,NaN,For Rent
1,2-BEDROOM APARTMENT,Shatti Al Qurum,2,2,500,NaN,For Rent
2,4+1 BEDROOM TWIN VILLA,Madinat Qaboos (MQ),4,5,1500,NaN,For Rent
3,COMMERCIAL SHOP,Ghala,NaN,NaN,370,39,For Rent
4,COMMERCIAL OFFICE SPACE,Bausher,NaN,2,4,NaN,For Rent


In [4]:
print(hilal_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         173 non-null    object
 1   Location      168 non-null    object
 2   Bedrooms      137 non-null    object
 3   Bathrooms     138 non-null    object
 4   Price         173 non-null    int64 
 5   Size          67 non-null     object
 6   Listing_Type  173 non-null    object
dtypes: int64(1), object(6)
memory usage: 9.6+ KB
None


In [5]:
print("\nDubizzle Dataset:")
dubizzle_df.head()


Dubizzle Dataset:


,property_name,price,location,area,bathrooms,beds
0,MADINAT AL ILAM | EXCELLENT 4+1 BR TOWNHOUSE,OMR 650,"Madinat As Sultan Qaboos, Muscat•",300 SQM,6,4
1,AL KHUWAIR | WELL MAINTAINED 2 BHK FLAT,OMR 280,"Al Khuwair, Muscat•",105 SQM,2,2
2,Elegant 4 BHK Villa for rent @ Qurum,OMR 650,"Qurum, Muscat•",300 SQM,5,4
3,ANSAB | 8 BEDROOM VILLA,OMR 400,"Ansab, Muscat•",300 SQM,9,8
4,ADV905*4BHK Villa for rent in Madinat Illam in...,OMR 750,"Qurum, Muscat•",300 SQM,4,4


In [6]:
print(dubizzle_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   property_name  9360 non-null   object
 1   price          9360 non-null   object
 2   location       9360 non-null   object
 3   area           9360 non-null   object
 4   bathrooms      9360 non-null   object
 5   beds           9360 non-null   object
dtypes: object(6)
memory usage: 438.9+ KB
None


## 4. Standardize column names for both datasets

In [7]:
hilal_df = hilal_df.rename(columns={
    'Title': 'property_name',
    'Location': 'location',
    'Bedrooms': 'bedrooms',
    'Bathrooms': 'bathrooms',
    'Price': 'price',
    'Size': 'size',
    'Listing_Type': 'listing_type'
})

dubizzle_df = dubizzle_df.rename(columns={
    'property_name': 'property_name',
    'price': 'price',
    'location': 'location',
    'area': 'size',
    'bathrooms': 'bathrooms',
    'beds': 'bedrooms'
})

## 5. Combine the datasets

In [8]:
df = pd.concat([hilal_df, dubizzle_df], ignore_index=True)

## 6. Trim whitespace from strings

In [9]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip()

## 7. Clean numeric columns: price, bedrooms, bathrooms

In [10]:
for col in ['price', 'bedrooms', 'bathrooms']:
    df[col] = df[col].replace(r"[^\d]", "", regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

## 8. Clean and Standardize Property Size

In [17]:
# Remove 'SQM' or any non-digit characters
df['size'] = df['size'].replace(r"[^\d.]", "", regex=True)

# Convert to numeric
df['size'] = pd.to_numeric(df['size'], errors='coerce')

# Fill missing values with median
median_size = df['size'].median()
df['size'] = df['size'].fillna(median_size)

# Confirm no missing values
print("Missing in size column:", df['size'].isnull().sum())

Missing in size column: 0


## 9. Infer missing location from title text

In [18]:
known_locations = df['location'].dropna().unique().tolist()

def infer_location_from_title(row):
    if pd.isna(row['location']):
        for loc in known_locations:
            if isinstance(row['property_name'], str) and loc.lower() in row['property_name'].lower():
                return loc
    return row['location']

df['location'] = df.apply(infer_location_from_title, axis=1)
df['location'] = df['location'].fillna('Unknown')

## 10. Fill missing bedrooms & bathrooms by location median

In [28]:
median_bedroom = df['bedrooms'].median()
df['bedrooms'] = df['bedrooms'].fillna(round(median_bedroom)).astype("Int64")
median_bathroom = df['bathrooms'].median()
df['bedrooms'] = df['bathrooms'].fillna(round(median_bathroom)).astype("Int64")


## 11. Convert to integers again after fill

In [29]:
for col in ['price', 'bedrooms', 'bathrooms']:
    df[col] = df[col].replace(r"[^\d.]", "", regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce').round().astype("Int64")

df['bedrooms'] = df['bedrooms'].round().astype("Int64")
df['bathrooms'] = df['bathrooms'].round().astype("Int64")
df['price'] = df['price'].round().astype("Int64")

## 12. Drop duplicates and listing_type

In [30]:
df = df.drop_duplicates()
df = df.drop(columns=['listing_type'], errors='ignore')

## 10. View & Export the Combined Datase

In [33]:
df.to_csv("Merged_Cleaned_Dataset.csv", index=False)
df.head()

,property_name,location,bedrooms,bathrooms,price,size
0,2-BEDROOM APARTMENT,Al Ansab,2,2,300,120.0
1,2-BEDROOM APARTMENT,Shatti Al Qurum,2,2,500,120.0
2,4+1 BEDROOM TWIN VILLA,Madinat Qaboos (MQ),5,5,1500,120.0
3,COMMERCIAL SHOP,Ghala,2,<NA>,370,39.0
4,COMMERCIAL OFFICE SPACE,Bausher,2,2,4,120.0


In [32]:
df.tail()

,property_name,location,bedrooms,bathrooms,price,size
9528,Room for bachelor,"Azaiba, Muscat•",1,1,100,10.0
9529,Sharing room,"Al Ghubrah, Muscat•",2,2,60,20.0
9530,شقق وغرف للايجار,"Al Khuwair, Muscat•",1,1,200,50.0
9531,شقة للايجار الدور الثاني العامرات… (للتواصل وا...,"Al Amarat, Muscat•",3,3,130,131.0
9532,صلالة صحنوت,"Salalah, Dhofar•",5,5,140,300.0
